In [56]:
from typing import NamedTuple
from kfp.v2 import dsl
from kfp.v2.dsl import (Artifact,
                        Dataset,
                        Input,
                        Model,
                        Output,
                        Metrics,
                        ClassificationMetrics,
                        component, 
                        OutputPath, 
                        InputPath)

from kfp.v2 import compiler
# from google.cloud import aiplatform
from google.cloud.aiplatform import pipeline_jobs

# PATH=%env PATH
# %env PATH={PATH}:/home/jupyter/.local/bin

# REGION="us-central1"

# PROJECT_ID= "udemy-mlops"

# BUCKET_NAME = "gs://sid-ml-ops"

# PIPELINE_ROOT = f"{BUCKET_NAME}/bikeshare-kfpl/"

# USER_FLAG = "--user"

In [52]:
from kfp.v2.dsl import component, OutputPath
from typing import NamedTuple

@component(
packages_to_install=["google-cloud-aiplatform","gcsfs","scikit-learn","pandas","google-cloud-storage","gsutil"]
)
def custom_training_job_component(
    output_path:Output[Artifact]
):   
    from google.cloud import aiplatform
    from google.cloud import storage
    
    aiplatform.init(project="udemy-mlops", location="us-central1", staging_bucket="gs://sid-ml-ops")
    
    bucket = "sid-ml-ops"
    source_blob_name = "model-training.py"
    
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket)
    blob = bucket.blob(source_blob_name)
    blob.download_to_filename("model-training.py")
    
    job = aiplatform.CustomTrainingJob(
        display_name="bikeshare-training-job",
        script_path="model-training.py",
        container_uri="us-docker.pkg.dev/vertex-ai/training/scikit-learn-cpu.0-23:latest",
        requirements=["gcsfs"]
    )
    
    job.run(
        replica_count=1,
        machine_type="n1-standard-4",
        sync=True
    )

    job.wait()
    output_path = "gs://sid-ml-ops/bike-share-rf-regression-artifact/"


In [53]:
@component(
    packages_to_install=["google-cloud-aiplatform"]
)
def deploy_model_component()-> NamedTuple("endpoint", [("url", str)]):
    
    from google.cloud import aiplatform
    
    aiplatform.init(project="udemy-mlops", location="us-central1", staging_bucket="gs://sid-ml-ops")
    
    model = aiplatform.Model.upload(
        display_name="bikeshare-model",
        artifact_uri="gs://sid-ml-ops/bike-share-rf-regression-artifact/",
        serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-0:latest",
        sync=False
    )
    
    DEPLOYED_NAME = "bikeshare-ep-new"
    TRAFFIC_SPLIT = {"0": 100}
    MIN_NODES = 1
    MAX_NODES = 1

    endpoint = model.deploy(
        deployed_model_display_name=DEPLOYED_NAME,
        traffic_split=TRAFFIC_SPLIT,
        machine_type="n1-standard-4",
        min_replica_count=MIN_NODES,
        max_replica_count=MAX_NODES
    )

In [54]:
@dsl.pipeline(
    pipeline_root="gs://sid-ml-ops/bikeshare-pipeline-root",
    name="bikeshare-kubeflow-pipeline",   
)
def pipeline(
    project: str = "udemy-mlops",
    region: str = "us-central1", 
    display_name: str = "bikeshare-pipeline"
    ):
    model_training = custom_training_job_component()
    endpoint = deploy_model_component().after(model_training)

In [ ]:
compiler.Compiler().compile(pipeline_func=pipeline,package_path='bikeshare-pipeline.json')

start_pipeline = pipeline_jobs.PipelineJob(
    display_name="bikeshare-pipeline",
    template_path="bikeshare-pipeline.json",
    enable_caching=False,
    location="us-central1",
)

start_pipeline.run()

Creating PipelineJob
PipelineJob created. Resource name: projects/1090925531874/locations/us-central1/pipelineJobs/bikeshare-kubeflow-pipeline-20230710074923
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/1090925531874/locations/us-central1/pipelineJobs/bikeshare-kubeflow-pipeline-20230710074923')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/bikeshare-kubeflow-pipeline-20230710074923?project=1090925531874
PipelineJob projects/1090925531874/locations/us-central1/pipelineJobs/bikeshare-kubeflow-pipeline-20230710074923 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/1090925531874/locations/us-central1/pipelineJobs/bikeshare-kubeflow-pipeline-20230710074923 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/1090925531874/locations/us-central1/pipelineJobs/bikeshare-kubeflow-pipeline-20230710074923 current state:
PipelineState.PIPELINE_STATE_R